In [1]:
import os
import sys
import pickle
import csv
import copy

import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
sys.path.append("../../../tfobjdetect/lib")
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

../../../tfobjdetect/lib/object_detection/utils/visualization_utils.py:25: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  import matplotlib; matplotlib.use('Agg')  # pylint: disable=multiple-statements


## Loading and preparing label maps

In [5]:
with open('../../../wsod/metadata/ont_m18/class_names_all.pkl', 'rb') as fin:
    mid2name_all = pickle.load(fin)

In [7]:
with open('../../results/det_results_concat_25.pkl', 'rb') as fin:
    det_results_concat = pickle.load(fin)

In [8]:
with open('../../results/det_results_merged_25.pkl', 'rb') as fin:
    det_results_merged = pickle.load(fin)

In [9]:
with open('../../results/det_results_postproc_25.pkl', 'rb') as fin:
    det_results_postproc = pickle.load(fin)

## Choosing examples for each class to visualize

In [12]:
label_to_img = {key: [] for key in mid2name_all}
for key, val in det_results_postproc.items():
    for det in val:
        if det['score'] >= 0.1:
            label_to_img[det['label']].append(key)

In [15]:
all_labels = list(label_to_img.keys())

In [16]:
sort_idx = np.argsort([-len(label_to_img[label]) for label in all_labels])

In [17]:
for i in sort_idx:
    print(all_labels[i], mid2name_all[all_labels[i]], len(label_to_img[all_labels[i]]))

/m/01g317 PER (Person) 42150
/m/04yx4 PER (Man) 10795
/m/0k4j VEH.WheeledVehicle.Car (Car) 5211
/m/07yv9 VEH (Vehicle) 3623
/m/0cgh4 FAC.Building (Building) 2796
/m/03bt1vf PER (Woman) 2553
/m/0hf58v5 COM (Luggage_and_bags) 2122
/m/01prls VEH.WheeledVehicle (Land_vehicle) 1487
/m/03120 COM.Flag.Flag (Flag) 1425
/m/07k1x COM.Equipment (Tool) 1387
/m/0bs7_0t COM (Electronic_device) 1375
/m/0cmf2 VEH.Aircraft.Airplane (Airplane) 1143
/m/04hgtk PER (Human_head) 1087
/m/019jd VEH.Watercraft.Boat (Boat) 1063
/m/07r04 VEH.WheeledVehicle.Truck (Truck) 956
syn_001 PER.ProfessionalPosition.Spokesperson (Orator) 836
/m/07cx4 COM (Telephone) 775
/m/07cmd VEH.MilitaryVehicle.Tank (Tank) 739
/m/03jm5 FAC.Building.House (House) 721
/m/083kb WEA (Weapon) 719
/m/07jdr VEH.WheeledVehicle.Train (Train) 577
/m/0hg7b COM.Equipment (Microphone) 451
/m/04_tb COM.Document.Map (Map) 446
syn_018 Contact.PublicStatementInPerson.Broadcast (Public_speaking) 402
syn_004 ORG.MilitaryOrganization.GovernmentArmedForce

In [70]:
select_labels = []
for l in all_labels:
    if mid2name_all[l].startswith('WEA'):
        select_labels.append(l)

In [71]:
select_labels

['/m/04ctx',
 '/m/083kb',
 '/m/04ylt',
 '/m/06nrc',
 '/m/06y5r',
 '/m/0gxl3',
 '/m/02gzp',
 '/m/020kz',
 '/m/06c54',
 '/m/0jb3',
 '/m/01vzwd',
 '/m/034qg',
 '/m/04zjc',
 '/m/02bb1s',
 '/m/065zrn9',
 '/m/032rk',
 '/m/0728h',
 '/m/0ct4f',
 '/m/05t_pq',
 '/m/06fk8',
 'syn_006',
 'syn_008',
 'syn_017']

In [24]:
#select_labels = ['/m/01fnck', '/m/019jd', '/m/03120', '/m/02_41', '/m/07cmd', '/m/06q40', '/m/03qtwd', '/m/09x0r', '/m/01xgg_', '/m/08qrwn', '/m/012n4x', '/m/02p16m6', '/m/02lbcq', '/m/09ct_', '/m/0g54v5d', '/m/0bg2p', '/m/0ct4f', '/m/01nd_n', '/m/09rvcxw', '/m/01nl4x', '/m/04ctx', '/m/04ylt', '/m/0gvss07', '/m/01rzcn', '/m/06nrc', '/m/0cyfs', '/m/0f5lx', '/m/04zjc', '/m/0dhz0', '/m/01lcw4', '/m/03htg', '/m/0gxl3', '/m/01bq8v', '/m/0lt4_', '/m/02gzp', '/m/02yjc', '/m/012n7d', '/m/0jb3', '/m/0dwx7']
select_labels = [all_labels[i] for i in sort_idx]

## Visualizing Results

In [18]:
mid2name_extended = {}
extended_classes = set()
for key in all_labels:
    for model, abv in [('coco', 'CO'), ('oi', 'OI'), ('ws', 'WS'), ('pascal','PA'), ('coco/J', 'CO/J'), ('oi/J', 'OI/J'), ('ws/J', 'WS/J'), ('pascal/J','PA/J')]:
        extended_classes.add(key + '/' + model)
        mid2name_extended[key + '/' + model] = mid2name_all[key] + f' ({abv})'
extended_classes = list(extended_classes)
mid2idx_extended = {}
for i, key in enumerate(extended_classes):
    mid2idx_extended[key] = i
category_index_extended = {mid2idx_extended[key]:{'id': mid2idx_extended[key], 'name': mid2name_extended[key]} for key in mid2name_extended}

In [19]:
def preproc(im):
    target_size = 400
    max_size = 1024
    im_size_min = np.min(im.shape[0:2])
    im_size_max = np.max(im.shape[0:2])
    im_scale = float(target_size) / float(im_size_min)
    # Prevent the biggest axis from being more than MAX_SIZE
    if np.round(im_scale * im_size_max) > max_size:
        im_scale = float(max_size) / float(im_size_max)
    im = cv2.resize(im, None, None, fx=im_scale, fy=im_scale,
                    interpolation=cv2.INTER_LINEAR)    
    return im

In [20]:
result_dict = {
    'Not Merged': det_results_concat,
    'Merged': det_results_merged,
    'Post-Processed': det_results_postproc,
}

In [21]:
path_prefix = '../../../../data/eval_m9/jpg/' 

In [22]:
model_abv_dict = {
    'coco': 'CO',
    'pascal': 'PA',
    'oi': 'OI',
    'ws': 'WS',
    'coco/J': 'CO/J',
    'pascal/J': 'PA/J',
    'oi/J': 'OI/J',
    'ws/J': 'WS/J',
}

In [23]:
def show(imgid, source, thresh):
    filename = path_prefix + imgid + '.jpg.ldcc'
    print(imgid)
    with open(filename, 'rb') as fin:
        _ = fin.read(1024)
        imgbin = fin.read()
    imgbgr = cv2.imdecode(np.fromstring(imgbin, dtype='uint8'), cv2.IMREAD_COLOR)
    image_np = imgbgr[:,:,[2,1,0]]
    image_np = preproc(image_np)
    
    detections = result_dict[source][imgid]
    boxes = np.asarray([det['bbox_normalized'] for det in detections])[:,[1,0,3,2]]
    scores = [det['score'] for det in detections]
    label_idx = [mid2idx_extended[det['label'] + '/' + det['model']] for det in detections]
    
    # print([(mid2name_extended[extended_classes[item]], boxes[i]) for i, item in enumerate(label_idx) if scores[i] > thresh])
    
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      boxes,
      label_idx,
      scores,
      category_index_extended,
      use_normalized_coordinates=True,
      min_score_thresh=thresh,
      max_boxes_to_draw=1000,
      line_thickness=2)
    plt.figure(figsize=(12, 8))
    plt.imshow(image_np)
    plt.axis('off')
    plt.show()    

In [25]:
for label in select_labels:
    if len(label_to_img[label]) == 0:
        continue
    print(mid2name_all[label])
    imgid = np.random.choice(label_to_img[label])
    print(imgid)
    w = interactive(show, 
        imgid=fixed(imgid),
        source=widgets.RadioButtons(options=['Post-Processed', 'Merged', 'Not Merged'], value='Post-Processed'), 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.1, continuous_update=False), 
    )
    w.children[-1].layout.height = '600px'
    display(w)
    

PER (Person)


interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

PER (Man)


interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

VEH.WheeledVehicle.Car (Car)


...

interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

COM.Equipment (Radar)


interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

VEH.WheeledVehicle.FireApparatus (Fire_apparatus)


interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

In [76]:
for label in select_labels:
    if len(label_to_img[label]) == 0:
        continue
    print(mid2name_all[label])
    imgid = np.random.choice(label_to_img[label])
    print(imgid)
    w = interactive(show, 
        imgid=fixed(imgid),
        source=widgets.RadioButtons(options=['Post-Processed', 'Merged', 'Not Merged'], value='Post-Processed'), 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.1, continuous_update=False), 
    )
    w.children[-1].layout.height = '600px'
    display(w)
    

WEA (Weapon)
HC000XHBQ


interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

WEA.MissileSystem.Missile (Missile)
IC0011S93


interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

WEA.Gun.Firearm (Shotgun)
HC000RFK4


...

interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

WEA.DaggerKnifeSword.Hatchet (Hatchet)
IC0011MU0


interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

WEA.Bullets (Bullet)
IC001IOPZ


interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

In [27]:
query_docs = [  'HC00002ZO',
                'HC00002ZR',
                'HC00002ZQ',
                'HC000T67L',
                'HC00002ZN',
                'HC000T6B0',
                'HC000T69B',
                'HC000T6J5',
                'IC001JWOR',
                'HC000T6J3',
                'HC00002ZQ',
                'HC000T6IF',
                'HC00002ZS',
                'HC00002ZQ',
                'HC00002ZD',
                'HC000031G',
                'HC000T65T',
                'HC00002ZG',
                'HC000T6IT',
                'HC000T69A',
                'HC000Q7NJ',
                'HC000ZXRT',
                'HC000T65W',
                'HC0007DBW',
                'HC000Q6RT',
                'IC001JNOC',
                'IC001JNOL',
                'IC001L4P9',
                'HC000T6J5',
                'HC00038S5']

In [33]:
p2c = {}
c2p = {}
with open('../../../../data/eval_m9/parent_children.tab', 'r') as fin:
    flag = False
    for row in csv.reader(fin, delimiter='\t'):
        if not flag:
            flag = True
            continue
        if row[2] not in p2c:
            p2c[row[2]] = []
        p2c[row[2]].append(row[3])
        c2p[row[3]] = row[2]

In [34]:
parent_query_docs = set([c2p[item] for item in query_docs])

In [36]:
query_docs_expanded = set()
for item in parent_query_docs:
    query_docs_expanded |= set(p2c[item])

In [37]:
query_docs_expanded

{'HC00000GV',
 'HC00000GW',
 'HC00001FK',
 'HC00001FL',
 'HC00001FM',
 'HC00001FN',
 'HC00001FO',
 'HC00001FP',
 'HC00001FQ',
 'HC00001FR',
 'HC00001FS',
 'HC00001FT',
 'HC00001FU',
 'HC00001FV',
 'HC00001FW',
 'HC00001FX',
 'HC00001FY',
 'HC00001FZ',
 'HC00001G0',
 'HC00001G1',
 'HC00001G2',
 'HC00001G3',
 'HC00001G4',
 'HC00001G5',
 'HC00001G6',
 'HC00001G7',
 'HC00001G8',
 'HC00001G9',
 'HC00001GA',
 'HC00001GB',
 'HC00001GC',
 'HC00001GD',
 'HC00001GE',
 'HC00001GF',
 'HC00001GG',
 'HC00001GH',
 'HC00001GI',
 'HC00001GJ',
 'HC00001GK',
 'HC00001GL',
 'HC00001GM',
 'HC00001GN',
 'HC00001GO',
 'HC00001GP',
 'HC00001GQ',
 'HC00001GR',
 'HC00001GS',
 'HC00001GT',
 'HC00001GU',
 'HC00001GV',
 'HC00001GW',
 'HC00001GX',
 'HC00001GY',
 'HC00001GZ',
 'HC00001H0',
 'HC00001H1',
 'HC00001H2',
 'HC00001H3',
 'HC00001H4',
 'HC00001H5',
 'HC00001H6',
 'HC00001H7',
 'HC00001H8',
 'HC00001H9',
 'HC00001HA',
 'HC00001HB',
 'HC00001HD',
 'HC00001HE',
 'HC00001HF',
 'HC00001HG',
 'HC00001HH',
 'HC00

In [38]:
query_docs_expanded_jpg = set()
for item in query_docs_expanded:
    if item in det_results_postproc:
        query_docs_expanded_jpg.add(item)

In [39]:
query_docs_expanded_jpg

{'HC00001FK',
 'HC00001FL',
 'HC00001FM',
 'HC00001FN',
 'HC00001FO',
 'HC00001FP',
 'HC00001FQ',
 'HC00001FR',
 'HC00001FS',
 'HC00001FT',
 'HC00001FU',
 'HC00001FV',
 'HC00001FW',
 'HC00001FX',
 'HC00001FY',
 'HC00001FZ',
 'HC00001G0',
 'HC00001G1',
 'HC00001G2',
 'HC00001G3',
 'HC00001G4',
 'HC00001G5',
 'HC00001G6',
 'HC00001G7',
 'HC00001G8',
 'HC00001G9',
 'HC00001GA',
 'HC00001GB',
 'HC00001GC',
 'HC00001GD',
 'HC00001GE',
 'HC00001GF',
 'HC00001GG',
 'HC00001GH',
 'HC00001GI',
 'HC00001GJ',
 'HC00001GK',
 'HC00001GL',
 'HC00001GM',
 'HC00001GN',
 'HC00001GO',
 'HC00001GP',
 'HC00001GQ',
 'HC00001GR',
 'HC00001GS',
 'HC00001GT',
 'HC00001GU',
 'HC00001GV',
 'HC00001GW',
 'HC00001GX',
 'HC00001GY',
 'HC00001GZ',
 'HC00001H0',
 'HC00001H1',
 'HC00001H2',
 'HC00001H3',
 'HC00001H4',
 'HC00001H5',
 'HC00001H6',
 'HC00001H7',
 'HC00001H8',
 'HC00001H9',
 'HC00001HA',
 'HC00001HB',
 'HC00001HD',
 'HC00001HE',
 'HC00001HF',
 'HC00001HG',
 'HC00001HH',
 'HC00001HI',
 'HC00001HJ',
 'HC00

In [40]:
for imgid in query_docs_expanded_jpg:
    print(imgid)
    w = interactive(show, 
        imgid=fixed(imgid),
        source=widgets.RadioButtons(options=['Post-Processed', 'Merged', 'Not Merged'], value='Post-Processed'), 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.1, continuous_update=False), 
    )
    w.children[-1].layout.height = '600px'
    display(w)


IC0015UWL


interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

HC000SUQK


interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

HC00001GS


...

interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

HC00001LX


interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

HC00001PO


interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

In [41]:
'HC00001FQ' in det_results_postproc

True

In [42]:
imgid = 'HC00001FQ'
w = interactive(show, 
    imgid=fixed(imgid),
    source=widgets.RadioButtons(options=['Post-Processed', 'Merged', 'Not Merged'], value='Post-Processed'), 
    thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.1, continuous_update=False), 
)
w.children[-1].layout.height = '600px'
display(w)


interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

In [44]:
import xml.etree.ElementTree as ET
tree = ET.parse('../../temp/zerohop_queries.xml')
root = tree.getroot()

In [63]:
zerohop_docs = []
for child in root:
    if child[0][2][1].tag == 'topleft':
        print(child[0][2][0].text, child[0][1].text)
        zerohop_docs.append(child[0][2][0].text.strip())

 HC00001FQ   Person 
 HC00001FL   Person 
 IC00167BW   Person 
 IC0015Q45   Organization 
 IC0015O8C   Person 
 HC000ZNVV   Person 
 HC000ZNVS   Person 
 HC000ZNVT   Person 
 HC000SUP8   Person 
 IC001L462   Person 
 IC0015LAC   Organization 
 HC00001TG   Person 
 HC00001TH   Person 
 HC000SYGW   Person 
 IC0015LB2   Vehicle 


In [69]:
zerohop_docs_vid = []
for child in root:
    if len(child[0][2]) != 3:
        print(child[0][2][1].text, child[0][1].text)
        zerohop_docs_vid.append(child[0][2][1].text.strip())

 HC000SYGU_71   Person 
 HC000Q8MF_40   Organization 
 IC0019MX2_1   Vehicle 
 IC0015YFV_54   Vehicle 
 HC000SYGU_71   Vehicle 


In [67]:
for child in root:
    if child[0][2][1].tag == 'start':
        print(child[0][2][0].text, child[0][1].text, child[0][2][1].text, child[0][2][2].text)


 IC001L4MY   Person   398   414 
 HC000Q828   Person   234   250 
 HC000ZXSM   Person   971   986 
 HC000ZXSR   Person   664   676 
 IC001L4PA   Organization   1341   1356 
 HC00038S4   Person   826   834 
 IC001L4OD   Person   1663   1680 
 HC000ZXME   Person   310   327 
 HC000ZXML   Organization   805   820 
 HC000T6IT   Person   1767   1783 
 HC000T6CI   Person   666   678 
 HC000Q7NP   Organization   51   70 
 HC000T65W   Person   476   493 
 HC000ZXMV   Person   2   17 
 IC001L4O1   Person   132   151 
 HC000ZXN3   Person   314   333 
 HC000Q829   Organization   954   972 
 IC0015O8J   Organization   73   89 
 IC0015OAC   Person   1   17 
 HC000T6A1   Person   30   46 
 HC000T69I   Person   191   208 
 HC000T65D   Person   834   845 
 HC000Q82M   Person   243   253 
 HC000T694   Person   4902   4911 
 HC000T6IF   Person   154   164 
 IC001L4NQ   Person   1888   1896 
 IC001L4O4   Person   365   386 
 HC000Q7NJ   Person   828   879 
 HC000T6IT   Person   1381   1403 
 HC000T65O   

In [64]:
for imgid in zerohop_docs:
    print(imgid)
    w = interactive(show, 
        imgid=fixed(imgid),
        source=widgets.RadioButtons(options=['Post-Processed', 'Merged', 'Not Merged'], value='Post-Processed'), 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.1, continuous_update=False), 
    )
    w.children[-1].layout.height = '600px'
    display(w)


HC00001FQ


interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

HC00001FL


interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

IC00167BW


...

interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

HC000SYGW


interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

IC0015LB2


interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…